In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0027196407318115234
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 14.451986933885985
200000 27.621178080268947
300000 33.14479879455616
400000 35.91339346077926
500000 33.488802986595545
600000 34.08683340728596
700000 35.829194579454914
800000 33.75515647293583
900000 33.611986074463516
1000000 33.75104776851272
1100000 33.469512248464135
1200000 33.04826183877611
1300000 16.25919510443911
1400000 24.418098636619035
1500000 33.2869312144794
1600000 32.52147504278201
1700000 33.12230945042179
1800000 32.656550573350735
1900000 33.61520455717231
2000000 35.04725230721391
2100000 33.28470174605956
2200000 33.62654875608645
2300000 33.252655373657994
2400000 33.618853573025035
2500000 33.15789563486977
2600000 13.624867062903958
2700000 28.53951041130988
2800000 42.52407882826503
2900000 42.35094647016972
3000000 41.988371

29700000 29.252143767229143
29800000 31.493465260412165
29900000 31.586751642799573
30000000 32.156623121531865
30100000 32.21941654561458
30200000 31.82849280612936
30300000 32.05553538073997
30400000 33.89585716674207
30500000 32.97684872447444
30600000 31.937618025124795
30700000 12.726968986214988
30800000 34.707265536472654
30900000 34.24112448547876
31000000 33.801750611410455
31100000 20.426915907757337
31200000 24.533317422233253
31300000 34.142552203174525
31400000 34.3935844988776
31500000 34.14734364649601
31600000 32.89340165618074
31700000 34.03716952933857
31800000 33.205509453241056
31900000 32.27510384508282
32000000 32.15561325514556
32100000 31.27550221533797
32200000 7.374021775151987
32300000 32.27786752497739
32400000 31.177541026441535
32500000 32.19972942033638
32600000 32.96253257651008
32700000 32.667752877899524
32800000 31.68930248740559
32900000 33.541050049254075
33000000 33.30367485526627
33100000 32.80593691095924
33200000 20.651353593652583
33300000 25.0

59600000 15.895947534102705
59700000 23.8611661787638
59800000 29.475859787227336
59900000 28.165087046949896
60000000 28.356073701997193
60100000 27.34535404005086
60200000 26.54723194809132
60300000 26.325075462237237
60400000 22.691015105458057
60500000 11.704668827636175
60600000 26.083532686911262
60700000 26.252655620308094
60800000 26.90599750057968
60900000 27.456588613465865
61000000 28.079854593805972
61100000 28.04912349238148
61200000 28.572096869078724
61300000 13.718791708921774
61400000 27.295213617408127
61500000 29.70436378580485
61600000 29.00984016163025
61700000 30.791352985852516
61800000 30.12492071072982
61900000 30.097519481350655
62000000 30.40431593212397
62100000 30.36915286733577
62200000 13.629590622336726
62300000 26.147348219535903
62400000 29.160268294382295
62500000 28.567665828945177
62600000 29.249476104819408
62700000 27.837012393761164
62800000 27.171201787629077
62900000 27.065829098436488
63000000 16.419986682616553
63100000 18.2643537863301
63200

89300000 24.69382738792875
89400000 8.499065170578131
89500000 26.54800694676968
89600000 25.74550419057055
89700000 26.079146576812906
89800000 26.075249380447396
89900000 26.313223264957877
90000000 27.53337011578363
90100000 25.60418347237147
90200000 26.283189483190906
90300000 26.396597783465204
90400000 25.679246329702597
90500000 25.558290841069017
90600000 25.686908963707598
90700000 9.642481218822656
90800000 22.313941317041238
90900000 23.89189148515147
91000000 22.842181003898762
91100000 23.462607810198005
91200000 23.465837919251598
91300000 11.102372899572753
91400000 18.41355513154746
91500000 23.624877357558336
91600000 23.05169278126326
91700000 23.155016613324126
91800000 23.454844690685373
91900000 16.555586354994
92000000 17.820245641291084
92100000 24.704520051562962
92200000 24.708279699249733
92300000 25.83356532813595
92400000 25.253096286040048
92500000 26.06785500929849
92600000 25.525444781859452
92700000 26.759122470370208
92800000 26.22078527666541
92900000

118300000 22.584254918446547
118400000 23.87300696842712
118500000 23.308311110344906
118600000 23.586334418754916
118700000 23.61986211674591
118800000 22.153932125925518
118900000 21.86019172432242
119000000 21.30764010569177
119100000 21.576256471591634
119200000 6.539319095420945
119300000 21.60279669857159
119400000 21.698275975662526
119500000 20.894091909777078
119600000 7.23058533705745
119700000 19.611090216253853
119800000 20.888546595004453
119900000 21.256881636768593
120000000 11.479529480736861
120100000 17.911361501536916
120200000 21.0484847107655
120300000 21.060309385590507
120400000 21.568238749778875
120500000 22.03999936032722
120600000 22.441135761206812
120700000 23.2320959099465
120800000 22.46529521226414
120900000 23.10971978588859
121000000 23.14254101842877
121100000 22.938825188029988
121200000 22.408234184635095
121300000 22.528363113235155
121400000 21.05481280358844
121500000 21.708251194823845
121600000 20.366524781259866
121700000 21.229206539758355
12

...

232700000 7.659040585941047
232800000 6.715769206782923
232900000 11.803474015329787
233000000 12.232065825331546
233100000 9.180694792934244
233200000 5.649196453514496
233300000 12.014984049938574
233400000 11.317001113865256
233500000 12.22131913245288
233600000 12.523304617018287
233700000 12.042731890434505
233800000 12.98485291569208
233900000 12.884588181603574
234000000 12.557961970423703
234100000 12.684124211083345
234200000 11.97668206185952
234300000 12.679834171889695
234400000 12.428104495429029
234500000 10.876611584300864
234600000 11.706852480560919
234700000 10.50789587690961
234800000 11.683900940721344
234900000 3.8329491332490866
235000000 11.730971301971344
235100000 10.769359171665556
235200000 11.470831775731673
235300000 4.590427944420904
235400000 9.277151475647369
235500000 11.186440575181264
235600000 11.330243647150265
235700000 8.052407036791692
235800000 6.614915928538958
235900000 11.442000829021447
236000000 11.321174350218605
236100000 11.5453996620050

261500000 8.280090024864924
261600000 6.485304246378459
261700000 5.552905437478825
261800000 8.894475365263432
261900000 8.98391705217315
262000000 9.224652237062136
262100000 9.159605002296109
262200000 9.738512373096746
262300000 8.799375120567703
262400000 9.28803768771709
262500000 9.387851103572743
262600000 9.627191985407373
262700000 9.235258125487738
262800000 9.215285274333311
262900000 8.071254337895176
263000000 4.6066985036286825
263100000 8.442540418576405
263200000 8.720021629400987
263300000 8.49416780168824
263400000 8.54498819581785
263500000 8.87691681664845
263600000 2.447856077834685
263700000 8.346060439497876
263800000 8.93401664250116
263900000 8.840631695837182
264000000 8.714840723239055
264100000 8.921506325741571
264200000 5.8797696296433966
264300000 6.098893549939492
264400000 9.391037602244548
264500000 9.338568611880085
264600000 9.499248770366679
264700000 9.528212463988295
264800000 9.851479817260929
264900000 9.794377725494744
265000000 9.548339893688

290800000 6.138904877808296
290900000 6.44590865386951
291000000 4.68723613326205
291100000 4.323125147252703
291200000 6.220090555552798
291300000 6.097983103120613
291400000 6.052817483010296
291500000 5.923100461799649
291600000 5.828378595858286
291700000 5.841564318505438
291800000 5.9737897608486135
291900000 1.129975794790299
292000000 6.075465557325643
292100000 5.802435008076781
292200000 5.8622625537897415
292300000 5.687952290952588
292400000 6.282396682667901
292500000 6.062073233725608
292600000 6.5313336135018885
292700000 3.7878870527678443
292800000 4.931450315665653
292900000 6.419660250660323
293000000 6.579445014876358
293100000 6.805396200094261
293200000 6.276425532707141
293300000 6.609762952520398
293400000 6.467765568810403
293500000 6.279876309185666
293600000 2.0802862021732533
293700000 6.926615628536134
293800000 6.238846109159011
293900000 5.966929708746334
294000000 6.108248440728503
294100000 5.908998911291883
294200000 5.827065774797447
294300000 5.79463

320000000 3.3453951094049135
320100000 3.040904742295332
320200000 1.167700150591304
320300000 3.379622677847198
320400000 3.373905841426367
320500000 3.5272874186064764
320600000 3.55068371033183
320700000 3.718500635843168
320800000 3.5065074135632335
320900000 3.5879556953610248
321000000 3.647590898965322
321100000 3.5871030962097983
321200000 1.9424066761630596
321300000 2.859008642033565
321400000 3.623415298630397
321500000 3.4616211873278546
321600000 3.237070617588721
321700000 1.5150376704972266
321800000 3.4512916655192996
321900000 3.413078469665628
322000000 3.5249447139328898
322100000 3.2931026525191123
322200000 3.289221074663521
322300000 3.3518152881026584
322400000 3.2906963754364207
322500000 3.2345838512927383
322600000 3.0643683114544453
322700000 2.1089636296876764
322800000 1.91755122234589
322900000 3.072304268184356
323000000 3.260254343609097
323100000 3.2414680004686156
323200000 3.245564534985973
323300000 3.23032282751709
323400000 3.260088172994303
323500

348600000 0.6337256210271769
348700000 0.631145780508572
348800000 0.6209789249191853
348900000 0.6139473471708152
349000000 0.605022716966472
349100000 0.5764998941212979
349200000 0.597069361349384
349300000 0.5797322611724496
349400000 0.5707720499598052
349500000 0.5827962478138486
349600000 0.5251874514048994
349700000 0.25244091807624897
349800000 0.25701896606028213
349900000 0.4233322106099983
350000000 0.4009961973258886
350100000 0.4066839717247817
350200000 0.3923190291674853
350300000 0.37902985319434
350400000 0.41719120633744927
350500000 0.3716827866552115
350600000 0.3541017026598645
350700000 0.3211545290632373
350800000 0.34027665590573153
350900000 0.2929716499493884
351000000 0.10610449393759504
351100000 0.21913781358705958
351200000 0.31570491867021655
351300000 0.2706971302449452
351400000 0.25761161076545125
351500000 0.2918875486718217
351600000 0.2472486392855578
351700000 0.24163945547754764
351800000 0.23128624838360679
351900000 0.2248607522853732
352000000